In [1]:
from bs4 import BeautifulSoup
import bs4
from urllib.request import urlopen
from collections import defaultdict

In [8]:
def get_corrections(soup):
    comments_field = soup.find("div", {"id": "comments_and_corrections_field"})
    return comments_field.find_all("div", {"class": "correction_box"})

In [9]:
def get_corrected_sentence(p):
    if not p.find("span", {"class": "sline"}):
        return p.get_text().strip()
    else:
        sent = ""
        for comp in p.children:
            if type(comp) == bs4.element.NavigableString:
                sent += comp
            else:
                if not comp.find("span", {"class": "sline"}):
                    sent += comp.get_text().strip()
        return sent

In [10]:
def get_pair_sents(corrections):
    pair_sents = defaultdict(list)
    for cor in corrections:
        ref = cor.find("li", {"class": "corrected correct"})
        if ref:
            org = cor.find("li", {"class": "incorrect"}).get_text()
            ref = get_corrected_sentence(ref.find("p"))
            pair_sents[org].append(ref)
    return pair_sents

In [15]:
def download_parallel_sents(journal_id): 
    print(f"{'*'*20} {journal_id} {'*'*20}")
    url = f"https://lang-8.com/256735/journals/{journal_id}"
    soup = BeautifulSoup(urlopen(url), "html.parser")
    corrections = get_corrections(soup)
    return get_pair_sents(corrections)

In [16]:
download_parallel_sents("157470523741257021141724101236015610371")

******************** 157470523741257021141724101236015610371 ********************


defaultdict(list,
            {'Not a few athletes, however, may not be able to set foot on Japan’s soil.': ['A few athletes, however, may not be able to set foot on Japanese soil.'],
             'What matters is whether holding the games under the condition will still be worthwhile or not.': ['What matters is whether holding the games under these conditions will  be worthwhile or not.',
              'What matters is whether holding the games under these conditions will still be worthwhile or not.'],
             'Not a few athletes, however, must not be able to set foot on Japan’s soil.': ['Only a few athletes, however, would be able to set foot on Japan’s soil.']})